In [1]:
import pandas as pd
import numpy as np
import sys
from os import getcwd
from os.path import join
sys.path.insert(0, join(getcwd(), '..'))
import sentiment


DATASET_DIR = '../goemotions/data-v2/'

In [2]:
classes = sentiment.load_classes(DATASET_DIR)
classes

['admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise',
 'neutral']

In [3]:
import tensorflow as tf
from tensorflow.data import AUTOTUNE, Dataset


def read_to_ds(x_fname, y_fname, batch_size, prefetch_buffer):
    with open(x_fname, 'rb') as file:
        X = np.load(file, allow_pickle=True)
    with open(y_fname, 'rb') as file:
        y = np.load(file, allow_pickle=True)
    ds = Dataset.from_tensor_slices((X, y))
    if batch_size:
        ds = ds.batch(batch_size)
    if prefetch_buffer:
        ds = ds.prefetch(prefetch_buffer)
    return ds

In [4]:
%%time
BATCH_SIZE = 32

train_ds = read_to_ds('train_X.npy', 'train_y.npy', BATCH_SIZE, AUTOTUNE)
val_ds = read_to_ds('val_X.npy', 'val_y.npy', BATCH_SIZE, AUTOTUNE)
test_ds = read_to_ds('test_X.npy', 'test_y.npy', BATCH_SIZE, AUTOTUNE)

CPU times: total: 2.61 s
Wall time: 9.66 s


In [5]:
from tensorflow.keras.layers import (Dropout, Dense)
from tensorflow.keras import Sequential, regularizers

classifier = Sequential([
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=regularizers.L1(l1=0.01)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(len(classes), activation='sigmoid'),
])

In [6]:
from tensorflow.keras.losses import BinaryCrossentropy
from keras.optimizers import Adam

METRICS_THRESHOLD = 0.5
LEARNING_RATE = 1e-2


metrics = sentiment.create_metrics(classes, METRICS_THRESHOLD)
loss = BinaryCrossentropy()
classifier.compile(Adam(learning_rate=LEARNING_RATE), loss=loss, metrics=metrics)

In [7]:
from tensorflow.keras.callbacks import EarlyStopping


EPOCHS = 200


hist = classifier.fit(x=train_ds, validation_data=val_ds, epochs=EPOCHS,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=3)]
                      # class_weight=class_weights
                     )

Epoch 1/200
1122/1122 [==============================] - 74s 55ms/step - loss: 34.6234 - precision@0.5/admiration: 0.1282 - recall@0.5/admiration: 0.0012 - precision@0.5/amusement: 0.0769 - recall@0.5/amusement: 4.0634e-04 - precision@0.5/anger: 0.0000e+00 - recall@0.5/anger: 0.0000e+00 - precision@0.5/annoyance: 0.0833 - recall@0.5/annoyance: 7.4655e-04 - precision@0.5/approval: 0.3000 - recall@0.5/approval: 9.5027e-04 - precision@0.5/caring: 0.0182 - recall@0.5/caring: 8.6505e-04 - precision@0.5/confusion: 0.0250 - recall@0.5/confusion: 6.9930e-04 - precision@0.5/curiosity: 0.0000e+00 - recall@0.5/curiosity: 0.0000e+00 - precision@0.5/desire: 0.0909 - recall@0.5/desire: 0.0020 - precision@0.5/disappointment: 0.0000e+00 - recall@0.5/disappointment: 0.0000e+00 - precision@0.5/disapproval: 0.0000e+00 - recall@0.5/disapproval: 0.0000e+00 - precision@0.5/disgust: 0.0000e+00 - recall@0.5/disgust: 0.0000e+00 - precision@0.5/embarrassment: 0.0000e+00 - recall@0.5/embarrassment: 0.0000e+00 - 

KeyboardInterrupt: 

In [ ]:
eval_metrics = classifier.evaluate(x=test_ds, return_dict=True)
sentiment.print_metrics(eval_metrics)

In [ ]:
stats = hist.history
stats_graps = {
    'Val Loss': stats['val_loss'],
    'Val Precision All': stats['precision@0.5/all'],
    'Val Recall All': stats['recall@0.5/all'],
    'Val F1-Score Weighted': stats['f1_score@0.5/all'],
    'Val F1-Score Micro': stats['f1_score_micro@0.5/all'],
    'Val F1-Score Macro': stats['f1_score_macro@0.5/all'],
}
sentiment.plot_history(stats_graps)

In [ ]:
sentiment.calc_TP_perc(test_ds, classifier)

In [ ]:
import pickle

with open('vectorizer.pkl', 'rb') as file:
    vectorizer = pickle.load(file)
    
with open('features.pkl', 'rb') as file:
    features = pickle.load(file)

In [ ]:
examples = [
    'I am feeling great today!',
    'The weather is so good',
    'I have performed well at the university',
    'The war has started',
    'He is desperate in this cruel world',
    'I love the feeling when my girlfriend hugs me',
    'I hate monday mornings',
    'Look forward to seeing you today',
    'Merry Christmas! I told Santa you were good this year and '
    'asked him to bring you a year full of joy and pleasure ',
    'brilliant! Such a detailed review, it was a pleasure, thank you! '
    'Guys, make sure you find time to read :) Aaaaand you can actually choose sth new)',
    'I have the new pan for pancakes.',
    "I'm wearing a special red ribbon for luck.",
    'OMG, yep!!! That is the final answer! Thank you so much!',
    'I am so glad this is over',
    'Sorry, I feel bad for having said that',
    'Happy birthday, my friend! I wish you a lot of success!',
    'What a shame! I will never talk to him',
    "What if she knows? We don't know what to do",
    'WOW! I am really into cinema',
    "What if I don't pass the exam? I will never get this driving license!",
    'I have just come up with the idea of birthday present. Let me explain...',
    "Don't worry, all of us will pass this test. It is just to 'evaluate our knowledge.",
    ' miss my grandad. I am feeling so lonely after her death. '
    'I just lost my closest person..',
    'Skipping lessons is so miserable for Oxford students!',
    'Have a rest, my boy. You had a long trip. I will make us tea and bring a cake.',
    'What a man! My son got the highest rate and will study in Cambridge. '
    'Our family is so proud of him!',
    "Mmmm, delicious. That's totally the best pasta in Italy!",
]

In [ ]:
example_df = pd.DataFrame(examples, columns=['text'])
example_df = sentiment.cleansing(example_df)
example_df.head()

In [ ]:
exmpl_texts = example_df['text']
exmpl_vect = vectorizer.transform(exmpl_texts)
exmpl_features = exmpl_vect[:,features].toarray().astype('float32')

In [ ]:
model_scores = classifier(tf.constant(exmpl_features))
np_classes = np.array(classes)
for idx, predictions in enumerate(model_scores):
    predicted = (predictions >= 0.5).numpy()
    if predicted.any():
        emotions = np_classes[predicted]
    else:
        print('⚠️ Model not sure! ', end='')
        emotions = np_classes[[predictions.numpy().argmax()]]
    with_emojis = []
    for emotion in emotions:
        try:
            emoji = sentiment.EMOJI_MAP[emotion]
        except KeyError:
            emoji = 'N/A'
        with_emojis.append(f'{emotion} {emoji}')
    print('{}: {}'.format(exmpl_texts[idx], ' '.join(with_emojis)))

In [ ]:
sentiment.plot_conf_mtrx_all(classifier, test_ds, classes)

In [ ]:
sentiment.plot_conf_mtrx_per_class(classifier, test_ds, classes)

In [ ]:
classifier.save('./models/mlp', include_optimizer=False)

## Notes 


# Re-Run this notebook!!!  
# Implement emotion -> sentiment and check its accuracy